In [1]:
problem_txt = """
(define (problem logistics-test)
(:domain logistics)
(:objects
	apn4 apn5 apn2 apn8 apn3 apn6 apn7 apn1 - airplane
	cit5 cit6 cit3 cit1 cit2 cit4 - city
	apt4 apt5 apt6 apt8 apt7 apt3 apt2 apt1 - airport
	tru2 tru4 tru1 tru5 tru3 - truck
	obj23 obj44 obj13 obj12 obj22 obj11 obj88 obj99 obj33 obj77 obj21 obj66 obj55 obj00 - package
	pos33 pos55 pos21 pos23 pos11 pos22 pos66 pos44 pos77 pos13 pos12 - location
)
(:init
	(at apn4 apt4)
	(at apn5 apt4)
	(at apn2 apt2)
	(at apn8 apt1)
	(at apn3 apt6)
	(at apn6 apt2)
	(at apn7 apt8)
	(at apn1 apt3)
	(at obj23 pos12)
	(at obj44 pos23)
	(at obj13 pos21)
	(at obj12 pos22)
	(at obj22 pos23)
	(at obj11 pos21)
	(at obj88 pos22)
	(at obj99 pos33)
	(at obj33 pos21)
	(at obj77 pos66)
	(at obj21 pos55)
	(at obj66 pos22)
	(at obj55 pos23)
	(at obj00 pos11)
	(in-city apt4 cit1)
	(in-city apt5 cit4)
	(in-city apt6 cit2)
	(in-city apt8 cit6)
	(in-city apt7 cit3)
	(in-city apt3 cit5)
	(in-city apt2 cit1)
	(in-city apt1 cit4)
	(in-city pos33 cit3)
	(in-city pos55 cit2)
	(in-city pos21 cit6)
	(in-city pos23 cit4)
	(in-city pos11 cit2)
	(in-city pos22 cit6)
	(in-city pos66 cit5)
	(in-city pos44 cit1)
	(in-city pos77 cit3)
	(in-city pos13 cit1)
	(in-city pos12 cit6)
	(at tru3 pos33)
	(at tru5 pos55)
	(at tru1 pos22)
	(at tru4 pos23)
	(at tru2 pos66)
)
(:goal
	(and
		(at obj99 pos66)
		(at obj00 pos21)
		(at obj12 pos77)
		(at obj77 pos66)
))
)
"""

domain_txt = """
;; logistics domain Typed version.
;;

(define (domain logistics)
  (:requirements :strips :typing) 
  (:types truck
          airplane - vehicle
          package
          vehicle - physobj
          airport
          location - place
          city
          place 
          physobj - object)
  
  (:predicates 	(in-city ?loc - place ?city - city)
		(at ?obj - physobj ?loc - place)
		(in ?pkg - package ?veh - vehicle))
  
(:action LOAD-TRUCK
   :parameters    (?pkg - package ?truck - truck ?loc - place)
   :precondition  (and (at ?truck ?loc) (at ?pkg ?loc))
   :effect        (and (not (at ?pkg ?loc)) (in ?pkg ?truck)))

(:action LOAD-AIRPLANE
  :parameters   (?pkg - package ?airplane - airplane ?loc - place)
  :precondition (and (at ?pkg ?loc) (at ?airplane ?loc))
  :effect       (and (not (at ?pkg ?loc)) (in ?pkg ?airplane)))

(:action UNLOAD-TRUCK
  :parameters   (?pkg - package ?truck - truck ?loc - place)
  :precondition (and (at ?truck ?loc) (in ?pkg ?truck))
  :effect       (and (not (in ?pkg ?truck)) (at ?pkg ?loc)))

(:action UNLOAD-AIRPLANE
  :parameters    (?pkg - package ?airplane - airplane ?loc - place)
  :precondition  (and (in ?pkg ?airplane) (at ?airplane ?loc))
  :effect        (and (not (in ?pkg ?airplane)) (at ?pkg ?loc)))

(:action DRIVE-TRUCK
  :parameters (?truck - truck ?loc-from - place ?loc-to - place ?city - city)
  :precondition
   (and (at ?truck ?loc-from) (in-city ?loc-from ?city) (in-city ?loc-to ?city))
  :effect
   (and (not (at ?truck ?loc-from)) (at ?truck ?loc-to)))

(:action FLY-AIRPLANE
  :parameters (?airplane - airplane ?loc-from - airport ?loc-to - airport)
  :precondition
   (at ?airplane ?loc-from)
  :effect
   (and (not (at ?airplane ?loc-from)) (at ?airplane ?loc-to)))
)
"""

blocks_problem = open("blocks_problem.pddl", "w")
blocks_problem.write(problem_txt)
blocks_problem.close()

blocks_domain = open("blocks_domain.pddl", "w")
blocks_domain.write(domain_txt)
blocks_domain.close()

In [2]:
from unified_planning.engines.compilers.grounder import Grounder
from unified_planning.io.pddl_reader import PDDLReader
from unified_planning.engines import SequentialSimulator
from unified_planning.model import UPCOWState
from unified_planning.shortcuts import *
from typing import cast

Parsing e grounding del problema

In [3]:
reader = PDDLReader()
problem = reader.parse_problem('blocks_domain.pddl', 'blocks_problem.pddl')
problem = Grounder().compile(problem).problem
# Viene definito lo stato iniziale come un UPCOWState
init_state = UPCOWState(problem.initial_values)
# Viene inizializzato il simulatore integrato nel pddl
simulator = SequentialSimulator(problem)

In [5]:
from unified_planning.model.walkers import StateEvaluator

In [6]:
state_evaluator = StateEvaluator(problem)

In [7]:
hash(init_state) 

161437515090

In [8]:
type(problem.goals[0])

unified_planning.model.fnode.FNode

In [9]:
state_evaluator.evaluate(problem.goals[0], init_state)

false

Il simulatore restituisce una lista di eventi che rappresentano le possibili azioni che sono agibili da un determinato stato

In [10]:
events = simulator.get_applicable_events(init_state)
events = list(events)
for event in events:
  print(event.__dict__)

{'_conditions': [at(tru4, pos23), at(obj44, pos23)], '_effects': [at(obj44, pos23) := false, in(obj44, tru4) := true], '_simulated_effect': None}
{'_conditions': [at(tru1, pos22), at(obj12, pos22)], '_effects': [at(obj12, pos22) := false, in(obj12, tru1) := true], '_simulated_effect': None}
{'_conditions': [at(tru4, pos23), at(obj22, pos23)], '_effects': [at(obj22, pos23) := false, in(obj22, tru4) := true], '_simulated_effect': None}
{'_conditions': [at(tru1, pos22), at(obj88, pos22)], '_effects': [at(obj88, pos22) := false, in(obj88, tru1) := true], '_simulated_effect': None}
{'_conditions': [at(tru3, pos33), at(obj99, pos33)], '_effects': [at(obj99, pos33) := false, in(obj99, tru3) := true], '_simulated_effect': None}
{'_conditions': [at(tru2, pos66), at(obj77, pos66)], '_effects': [at(obj77, pos66) := false, in(obj77, tru2) := true], '_simulated_effect': None}
{'_conditions': [at(tru5, pos55), at(obj21, pos55)], '_effects': [at(obj21, pos55) := false, in(obj21, tru5) := true], '_sim

Per poter sapere quali azioni del problema originale siano eseguibili:

In [21]:
possible_actions = []
for ev in events:
    for ac in problem.actions:
        if ac.preconditions == ev.conditions and ac.effects == ev.effects:
            possible_actions.append(ac)
possible_actions[0]

action LOAD-TRUCK_obj44_tru4_pos23 {
    preconditions = [
      at(tru4, pos23)
      at(obj44, pos23)
    ]
    effects = [
      at(obj44, pos23) := false
      in(obj44, tru4) := true
    ]
    simulated effect = None
  }

Per valutare lo stato generato da una azione, utilizzare l'evento associato all'azione di interesse. Per esempio pick_up_d  

In [18]:
next_state = cast(UPCOWState, simulator.apply(events[0], init_state))
next_state

{at(obj44, pos23): false, in(obj44, tru4): true}
{at(apn4, apt4): true, at(apn5, apt4): true, at(apn2, apt2): true, at(apn8, apt1): true, at(apn3, apt6): true, at(apn6, apt2): true, at(apn7, apt8): true, at(apn1, apt3): true, at(obj23, pos12): true, at(obj44, pos23): true, at(obj13, pos21): true, at(obj12, pos22): true, at(obj22, pos23): true, at(obj11, pos21): true, at(obj88, pos22): true, at(obj99, pos33): true, at(obj33, pos21): true, at(obj77, pos66): true, at(obj21, pos55): true, at(obj66, pos22): true, at(obj55, pos23): true, at(obj00, pos11): true, in-city(apt4, cit1): true, in-city(apt5, cit4): true, in-city(apt6, cit2): true, in-city(apt8, cit6): true, in-city(apt7, cit3): true, in-city(apt3, cit5): true, in-city(apt2, cit1): true, in-city(apt1, cit4): true, in-city(pos33, cit3): true, in-city(pos55, cit2): true, in-city(pos21, cit6): true, in-city(pos23, cit4): true, in-city(pos11, cit2): true, in-city(pos22, cit6): true, in-city(pos66, cit5): true, in-city(pos44, cit1): true

In [28]:
events2 = simulator.get_applicable_events(next_state)
events2 = list(events2)
possible_actions2 = []
for ev in events2:
    for ac in problem.actions:
        if ac.preconditions == ev.conditions and ac.effects == ev.effects:
            possible_actions2.append(ac)
possible_actions2[0].name

'LOAD-TRUCK_obj12_tru1_pos22'

In [30]:
next_state2 = cast(UPCOWState, simulator.apply(events2[8], next_state))
next_state2

{in(obj44, tru4): false, at(obj44, pos23): true}
{at(obj44, pos23): false, in(obj44, tru4): true}
{at(apn4, apt4): true, at(apn5, apt4): true, at(apn2, apt2): true, at(apn8, apt1): true, at(apn3, apt6): true, at(apn6, apt2): true, at(apn7, apt8): true, at(apn1, apt3): true, at(obj23, pos12): true, at(obj44, pos23): true, at(obj13, pos21): true, at(obj12, pos22): true, at(obj22, pos23): true, at(obj11, pos21): true, at(obj88, pos22): true, at(obj99, pos33): true, at(obj33, pos21): true, at(obj77, pos66): true, at(obj21, pos55): true, at(obj66, pos22): true, at(obj55, pos23): true, at(obj00, pos11): true, in-city(apt4, cit1): true, in-city(apt5, cit4): true, in-city(apt6, cit2): true, in-city(apt8, cit6): true, in-city(apt7, cit3): true, in-city(apt3, cit5): true, in-city(apt2, cit1): true, in-city(apt1, cit4): true, in-city(pos33, cit3): true, in-city(pos55, cit2): true, in-city(pos21, cit6): true, in-city(pos23, cit4): true, in-city(pos11, cit2): true, in-city(pos22, cit6): true, in-ci

In [29]:
for i, action in enumerate(possible_actions2):
    if action.name == 'UNLOAD-TRUCK_obj44_tru4_pos23':
        print(i)

8


E' anche possibile generare un evento da una azione qualsiasi. Prendiamo direttamente l'azione 'pick_up_d' dalla lista delle azioni. Eseguiamolo. Se lo stato che otteniamo non è None allora vuol dire che l'azione può essere eseguita e genera quello stato

In [40]:
next_state2._values

{in(obj44, tru4): false, at(obj44, pos23): true}

In [14]:
print(next_state)
print()
print(init_state)

{at(obj44, pos23): false, in(obj44, tru4): true}
{at(apn4, apt4): true, at(apn5, apt4): true, at(apn2, apt2): true, at(apn8, apt1): true, at(apn3, apt6): true, at(apn6, apt2): true, at(apn7, apt8): true, at(apn1, apt3): true, at(obj23, pos12): true, at(obj44, pos23): true, at(obj13, pos21): true, at(obj12, pos22): true, at(obj22, pos23): true, at(obj11, pos21): true, at(obj88, pos22): true, at(obj99, pos33): true, at(obj33, pos21): true, at(obj77, pos66): true, at(obj21, pos55): true, at(obj66, pos22): true, at(obj55, pos23): true, at(obj00, pos11): true, in-city(apt4, cit1): true, in-city(apt5, cit4): true, in-city(apt6, cit2): true, in-city(apt8, cit6): true, in-city(apt7, cit3): true, in-city(apt3, cit5): true, in-city(apt2, cit1): true, in-city(apt1, cit4): true, in-city(pos33, cit3): true, in-city(pos55, cit2): true, in-city(pos21, cit6): true, in-city(pos23, cit4): true, in-city(pos11, cit2): true, in-city(pos22, cit6): true, in-city(pos66, cit5): true, in-city(pos44, cit1): true

In [30]:
for i in range(len(problem.actions)):
    pick_up_d_event = list(simulator.get_events(problem.actions[i],[]))[0]
    state_pick_up_d_event = cast(UPCOWState, simulator.apply(pick_up_d_event, init_state))
    if state_pick_up_d_event:
        print(i)

58
59
98
104
110
133
139
145
168
174
180
384
386
388


In [31]:
pick_up_d_event = list(simulator.get_events(problem.actions[58],[]))[0]
state_pick_up_d_event = cast(UPCOWState, simulator.apply(pick_up_d_event, init_state))
state_pick_up_d_event

{at(apn1, apt1): false, at(apn1, apt3): true}
{at(apn1, apt1): true, at(tru1, pos1): true, at(obj11, pos1): true, at(obj12, pos1): true, at(obj13, pos1): true, at(tru2, pos2): true, at(obj21, pos2): true, at(obj22, pos2): true, at(obj23, pos2): true, at(tru3, pos3): true, at(obj31, pos3): true, at(obj32, pos3): true, at(obj33, pos3): true, in-city(tru1, pos1, cit1): true, in-city(tru1, apt1, cit1): true, in-city(tru2, pos2, cit2): true, in-city(tru2, apt2, cit2): true, in-city(tru3, pos3, cit3): true, in-city(tru3, apt3, cit3): true, at(tru1, pos2): false, at(tru3, pos2): false, at(pos2, pos2): false, at(pos3, pos2): false, at(pos1, pos2): false, at(apn1, pos2): false, at(apt3, pos2): false, at(apt2, pos2): false, at(apt1, pos2): false, at(obj33, pos2): false, at(obj32, pos2): false, at(obj31, pos2): false, at(cit1, pos2): false, at(cit2, pos2): false, at(cit3, pos2): false, at(obj11, pos2): false, at(obj13, pos2): false, at(obj12, pos2): false, at(tru1, pos3): false, at(tru2, pos3): f

Nel caso l'azione non possa essere eseguita avremo None. Prendiamo l'azione put_down_d

In [9]:
put_down_d_event = list(simulator.get_events(problem.actions[4],[]))[0]
state_put_down_d_event = cast(UPCOWState, simulator.apply(put_down_d_event, init_state))
print(state_put_down_d_event is None)

True


In [ ]:
target_action_string = "DRIVE-TRUCK_tru2_pos55_pos33_cit3"
starting_action_string = "drivetrucktru2pos55pos33cit3"

In [ ]:
tmp = starting_action_string.replace("drivetruck", "DRIVE-TRUCK ")
tmp

'drivetrucktru2pos55pos33cit3'

In [ ]:
action, objects_string = tmp.split()
action, objects_string

('DRIVE-TRUCK', 'tru2pos55pos33cit3')

In [ ]:
import re

In [ ]:
objects = re.findall("[a-z]+\\d+", objects_string)
objects

['tru2', 'pos55', 'pos33', 'cit3']

In [ ]:
objects.insert(0, action)
objects

['DRIVE-TRUCK', 'tru2', 'pos55', 'pos33', 'cit3']

In [ ]:
action_name = "_".join(objects)
action_name

'DRIVE-TRUCK_tru2_pos55_pos33_cit3'

In [ ]:
for action in possible_actions:
    if action.name == action_name:
        print(action)
        break

action DRIVE-TRUCK_tru2_pos55_pos33_cit3 {
    preconditions = [
      at(tru2, pos55)
    ]
    effects = [
      at(tru2, pos55) := false
      at(tru2, pos33) := true
    ]
    simulated effect = None
  }


In [ ]:
possible_actions

In [ ]:
events[0].__dict__

{'_conditions': [at(tru5, pos12), at(obj21, pos12)],
 '_effects': [at(obj21, pos12) := false, in(obj21, tru5) := true],
 '_simulated_effect': None}

In [ ]:
starting_action_string = "DRIVE-TRUCK_tru2_pos55_pos33_cit3"
target_action_string = "drivetrucktru2pos55pos33cit3"

In [ ]:
tmp = starting_action_string.lower()
tmp

'drive-truck_tru2_pos55_pos33_cit3'

In [ ]:
parts = tmp.split("-")
parts = [parts[0]] + parts[1].split("_")
parts

['drive', 'truck', 'tru2', 'pos55', 'pos33', 'cit3']

In [ ]:
"".join(parts) == target_action_string

True

In [ ]:
def reverse_format(action_name):
    tmp = action_name.lower()
    tmp = tmp.split("-")
    tmp = [tmp[0]] + tmp[1].split("_")
    return "".join(tmp)

In [ ]:
reverse_format(starting_action_string) == target_action_string

True

In [ ]:
init_state

{at(apn3, apt4): true, at(apn8, apt8): true, at(obj23, pos77): true, at(obj99, pos77): true, at(obj21, pos12): true, at(obj44, pos55): true, at(obj11, pos23): true, at(obj00, pos55): true, in-city(apt8, cit6): true, in-city(apt4, cit3): true, in-city(pos77, cit6): true, in-city(pos33, cit3): true, in-city(pos44, cit3): true, in-city(pos23, cit3): true, in-city(pos12, cit6): true, in-city(pos55, cit3): true, at(tru5, pos12): true, at(tru2, pos55): true, in-city(apt8, cit3): false, in-city(pos77, cit3): false, in-city(pos12, cit3): false, in-city(apt4, cit6): false, in-city(pos33, cit6): false, in-city(pos44, cit6): false, in-city(pos23, cit6): false, in-city(pos55, cit6): false, at(apn3, apt8): false, at(tru2, apt8): false, at(tru5, apt8): false, at(obj23, apt8): false, at(obj99, apt8): false, at(obj21, apt8): false, at(obj44, apt8): false, at(obj11, apt8): false, at(obj00, apt8): false, at(apn8, apt4): false, at(tru2, apt4): false, at(tru5, apt4): false, at(obj23, apt4): false, at(obj9

In [ ]:
problem.goals[0].args[0].fluent()

bool at[obj=physobj, loc=place]

In [ ]:
init_state.__dict__['_values'][problem.goals[0].args[1]]

false